- i have devided age-group(3+) number by whole india/state pop, 
- I also could have devided by total sum col of 3+ language people 
- but which one is being asked
- should I report statePop devided ratios or totalPop(3+) devided ratios
- at least this is clear that the age-group we get won't be affected as highest ratio is scale invariante
- I am getting `30-49` age group for all states
- if we don't use statePop than census data won't be needed
- also que-6 is same with different dataset
- should I use state/ut name of just stateCode in these questions?

In [70]:
import pandas as pd

In [71]:
# for population data
cols=['State', 'Level', 'Name', 'TRU', 'No_HH', 'TOT_P', 'TOT_M', 'TOT_F']
census=pd.read_excel('datasets/census.xlsx',engine='openpyxl',usecols=cols)
censusIndia=census.iloc[0,:]
census=census.loc[census.Level=='STATE']
census=census.loc[census.TRU=='Total']
census=census.append(censusIndia,ignore_index=True)
census.sort_values(by=['State'],axis=0,inplace=True)
census.reset_index(drop=True,inplace=True)

In [72]:
#===================
# # a function that just takes state code that does
# manipulates and finds appropriate sum
# finds total pop fro that state from census data
# then returns a nice ratio dictionary
#===================

def ratioFinder(stateCode):
    # 5 is specific to TOT_P
    statePop=census.iloc[stateCode,5] 
    df=pd.read_excel('datasets/C-18.xlsx',skiprows=6,header=None,engine='openpyxl')
    df=df.loc[df.iloc[:,0]==stateCode]
    # also can use this to devide rather than statePop
    # the order should not change
    totalPop=int(df.loc[df.iloc[:,3]=='Total'].iloc[0,8])
    #print(totalPop)
    df=df.loc[df.iloc[:,3]=='Total'].iloc[1:,:]
    ageList=df.iloc[:,4].tolist()
    highestRatio=0
    for i in range(len(ageList)):
        ageData={
            'state/ut':str(stateCode).zfill(2),
            'age-group':ageList[i],
            'percentage':(int((df.loc[df.iloc[:,4]==ageList[i]]).iloc[:,8])/totalPop)
        }
        if max(ageData['percentage'],highestRatio)>highestRatio:
            highestRatio=ageData['percentage']
            stateRatios=ageData
            #print(stateRatios)
    #print('Processed state -- ',stateCode)
    # to make more readable and comparable quickly rounded to 2 decimal points
    stateRatios['percentage']=round(stateRatios['percentage'],2)
    return stateRatios


In [73]:
%%time
ratioList=[ratioFinder(i) for i in range(0,36)]
ratioDF=pd.DataFrame(ratioList)
ratioDF.to_csv('outputs/age-india.csv',index=False)

CPU times: user 6.05 s, sys: 3.72 ms, total: 6.06 s
Wall time: 6.06 s
